<a href="https://colab.research.google.com/github/Lonely52Hz/ID2223_Scalable_Machine_Learning_and_Deep_Learning/blob/main/ID2223_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# set environment in colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
!pip install pyspark

In [15]:
import tensorflow as tf
from tensorflow import keras

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Colab").config("spark.ui.port", "4050").getOrCreate()
from pyspark.sql.functions import *

In [17]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [67]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models, losses, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import evaluation, util
import math
from datetime import datetime
import torch
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import sklearn
from scipy import stats

# Regression

In [19]:
# read files
train = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-train.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
test = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-test.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
dev = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-dev.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')

train.show(5)
train.printSchema()

+-------------+------+--------+-----+-----+--------------------+--------------------+
|        genre|  file|    year|index|score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+-----+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|  5.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|  3.8|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|  3.8|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|  2.6|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9| 4.25|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+-----+--------------------+--------------------+
only showing top 5 rows

root
 |-- genre: string (nullable = true)
 |-- file: string (nullable = true)
 |-- year: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- sentenc

In [20]:
# map [0, 5] to [-1, 1]
train = train.withColumn('score', col('score')/2.5 - 1)
test = test.withColumn('score', col('score')/2.5 - 1)
dev = dev.withColumn('score', col('score')/2.5 - 1)

train.show(5)
train.describe(['score']).show()

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|                 0.7|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

+-------+-------------------+
|summary|              score|
+-----

In [21]:
# find rows with missing value
train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()
test.select([count(when(col(c).isNull(), c)).alias(c) for c in test.columns]).show()
dev.select([count(when(col(c).isNull(), c)).alias(c) for c in dev.columns]).show()

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        6|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+



In [22]:
# delete rows with missing value
train = train.na.drop()
test = test.na.drop()
dev = dev.na.drop()

train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()
train.show(5)

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        0|
+-----+----+----+-----+-----+---------+---------+

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|     

In [23]:
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),\
                               pooling_mode_mean_tokens=True,\
                               pooling_mode_cls_token=False,\
                               pooling_mode_max_tokens=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [25]:
loss = losses.CosineSimilarityLoss(model=model)

In [26]:
# Transform the data so that it can be used to train and test
train_example = []
test_example = []
dev_example = []

df_train = train.toPandas()
df_test = test.toPandas()
df_dev = dev.toPandas()

for row in df_train.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[5], row[6]], label=float(row[4]))
  train_example.append(inp_example)

for row in df_test.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[5], row[6]], label=float(row[4]))
  test_example.append(inp_example)

for row in df_dev.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[5], row[6]], label=float(row[4]))
  dev_example.append(inp_example)

train_dataloader = DataLoader(train_example, shuffle=True, batch_size=16)

In [ ]:
# train

num_epochs = 1
model_save_path = 'output/training_stsbenchmark_'+'bert-base-uncased'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, name='sts-dev')
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up
model.fit(train_objectives=[(train_dataloader, loss)],
          optimizer_class=torch.optim.Adam,
          optimizer_params={'lr': 2e-5},
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=1000,
          output_path=model_save_path)

# to save time, we use the previously trained model instead of training it everytime

'\nnum_epochs = 1\nmodel_save_path = \'output/training_stsbenchmark_\'+\'bert-base-uncased\'+\'-\'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")\n\nevaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, name=\'sts-dev\')\nwarmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up\nmodel.fit(train_objectives=[(train_dataloader, loss)],\n          optimizer_class=torch.optim.Adam,\n          optimizer_params={\'lr\': 2e-5},\n          epochs=num_epochs,\n          warmup_steps=warmup_steps,\n          evaluator=evaluator,\n          evaluation_steps=1000,\n          output_path=model_save_path)\n'

In [ ]:
# built-in evaluation

#regression_model_path = '/content/drive/MyDrive/output/training_stsbenchmark_bert-base-uncased-2021-12-10_21-53-26'
#model = SentenceTransformer(regression_model_path)
model = SentenceTransformer(model_save_path)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_example, name='sts-test')
model.evaluate(test_evaluator)

0.5739843881319436

In [ ]:
# evaluation using Spearmean Correlation, they coincide with each other
sentencea = model.encode(df_test['sentenceA'])
sentenceb = model.encode(df_test['sentenceB'])
cos_sim = 1 - sklearn.metrics.pairwise.paired_cosine_distances(sentencea, sentenceb)
print(cos_sim)
result = stats.spearmanr(cos_sim, df_test['score'])
print(result)

[0.6670458  0.9073092  0.6678997  ... 0.07020611 0.38499272 0.02931237]
SpearmanrResult(correlation=0.5739843881319436, pvalue=2.127869343158135e-121)


# Classification

In [27]:
# load data from snli
c_train = spark.read.json('drive/MyDrive/ID2223_File/snli_1.0/snli_1.0_train.jsonl')
c_test = spark.read.json('drive/MyDrive/ID2223_File/snli_1.0/snli_1.0_test.jsonl')
c_dev = spark.read.json('drive/MyDrive/ID2223_File/snli_1.0/snli_1.0_dev.jsonl')
c_train.show(5)

+----------------+----------------+-------------+-------------------+--------------------+----------------------+--------------------+--------------------+----------------------+--------------------+
|annotator_labels|       captionID|   gold_label|             pairID|           sentence1|sentence1_binary_parse|     sentence1_parse|           sentence2|sentence2_binary_parse|     sentence2_parse|
+----------------+----------------+-------------+-------------------+--------------------+----------------------+--------------------+--------------------+----------------------+--------------------+
|       [neutral]|3416050480.jpg#4|      neutral|3416050480.jpg#4r1n|A person on a hor...|  ( ( ( A person ) ...|(ROOT (S (NP (NP ...|A person is train...|  ( ( A person ) ( ...|(ROOT (S (NP (DT ...|
| [contradiction]|3416050480.jpg#4|contradiction|3416050480.jpg#4r1c|A person on a hor...|  ( ( ( A person ) ...|(ROOT (S (NP (NP ...|A person is at a ...|  ( ( A person ) ( ...|(ROOT (S (NP (DT ...|


In [28]:
c_train=c_train.drop('_corrupt_record')
c_train=c_train.na.drop()
c_train=c_train.select('gold_label','sentence1','sentence2').withColumnRenamed('gold_label','label')
c_dev=c_dev.drop('_corrupt_record')
c_dev=c_dev.na.drop()
c_dev=c_dev.select('gold_label','sentence1','sentence2').withColumnRenamed('gold_label','label')
c_test=c_test.drop('_corrupt_record')
c_test=c_test.na.drop()
c_test=c_test.select('gold_label','sentence1','sentence2').withColumnRenamed('gold_label','label')
c_df_train=c_train.toPandas()
c_df_test=c_test.toPandas()
c_df_dev=c_dev.toPandas()
# get rid of '-'
c_df_train = c_df_train[~c_df_train['label'].isin(['-'])]
c_df_test = c_df_test[~c_df_test['label'].isin(['-'])]
c_df_dev = c_df_dev[~c_df_dev['label'].isin(['-'])]
print(c_df_train)

                label  ...                                          sentence2
0             neutral  ...  A person is training his horse for a competition.
1       contradiction  ...      A person is at a diner, ordering an omelette.
2          entailment  ...                  A person is outdoors, on a horse.
3             neutral  ...                  They are smiling at their parents
4          entailment  ...                         There are children present
...               ...  ...                                                ...
550147  contradiction  ...           four kids won awards for 'cleanest feet'
550148        neutral  ...  four homeless children had their shoes stolen,...
550149        neutral  ...  A man in a bodysuit is competing in a surfing ...
550150  contradiction  ...  A man in a business suit is heading to a board...
550151     entailment  ...  On the beautiful blue water there is a man in ...

[549367 rows x 3 columns]


In [29]:
# check if there are missing values
c_train.select([count(when(col(c).isNull(), c)).alias(c) for c in c_train.columns]).show()
c_test.select([count(when(col(c).isNull(), c)).alias(c) for c in c_test.columns]).show()
c_dev.select([count(when(col(c).isNull(), c)).alias(c) for c in c_dev.columns]).show()

+-----+---------+---------+
|label|sentence1|sentence2|
+-----+---------+---------+
|    0|        0|        0|
+-----+---------+---------+

+-----+---------+---------+
|label|sentence1|sentence2|
+-----+---------+---------+
|    0|        0|        0|
+-----+---------+---------+

+-----+---------+---------+
|label|sentence1|sentence2|
+-----+---------+---------+
|    0|        0|        0|
+-----+---------+---------+



In [30]:
c_train_example = []
c_test_example = []
c_dev_example = []

label2int = {"contradiction": 0, "neutral": 1, "entailment": 2}

for row in c_df_train.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[1], row[2]], label=label2int[row[0]])
  c_train_example.append(inp_example)

for row in c_df_test.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[1], row[2]], label=label2int[row[0]])
  c_test_example.append(inp_example)

for row in c_df_dev.iloc:
  row = row.values.tolist()
  inp_example = InputExample(texts=[row[1], row[2]], label=label2int[row[0]])
  c_dev_example.append(inp_example)

In [52]:
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),\
                               pooling_mode_mean_tokens=True,\
                               pooling_mode_cls_token=False,\
                               pooling_mode_max_tokens=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)

In [54]:
c_train_dataloader = DataLoader(c_train_example, shuffle=True, batch_size=16)

In [55]:
# train


num_epochs = 1
c_model_save_path = 'output/training_snil_'+'bert-base-uncased'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(c_dev_example, batch_size=16, name='snil-dev')
warmup_steps = math.ceil(len(c_train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up
model.fit(train_objectives=[(c_train_dataloader, loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=1000,
          output_path=c_model_save_path)

# to save time, we use the previously trained model instead of training it everytime

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/34336 [00:00<?, ?it/s]

In [63]:
#cp -rf /content/output/* /content/drive/MyDrive/output # save the trained model

In [56]:
# built-in evaluation

#classification_model_path = '/content/drive/MyDrive/output/training_snil_bert-base-uncased-2021-12-17_14-04-59'
#model = SentenceTransformer(classification_model_path)
model = SentenceTransformer(c_model_save_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_example, batch_size=16, name='snli-test')
#test_evaluator(model, output_path=classification_model_path)
test_evaluator(model, output_path=c_model_save_path)

0.7139430535720919

In [57]:
# evaluation using Spearmean Correlation, they coincide with each other
sentencea = model.encode(df_test['sentenceA'])
sentenceb = model.encode(df_test['sentenceB'])
cos_sim = 1 - sklearn.metrics.pairwise.paired_cosine_distances(sentencea, sentenceb)
print(cos_sim)
result = stats.spearmanr(cos_sim, df_test['score'])
print(result)

[0.92975056 0.9927016  0.961233   ... 0.5377749  0.7178428  0.48726493]
SpearmanrResult(correlation=0.7139430535720919, pvalue=6.56577697178156e-215)


# Combing both objectives


In [59]:
# load the classification model
#classification_model_path = '/content/drive/MyDrive/output/training_snil_bert-base-uncased-2021-12-17_14-04-59'
classification_model_path = c_model_save_path

In [60]:
# train


model = SentenceTransformer(classification_model_path)
train_dataloader = DataLoader(train_example, shuffle=True, batch_size=16)
loss = losses.CosineSimilarityLoss(model=model)
num_epochs = 1
combine_model_save_path = 'output/training_combined_'+'bert-base-uncased'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_example, name='combined-dev')
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) # 10% of train data for warm-up


model.fit(train_objectives=[(train_dataloader, loss)],
          optimizer_class=torch.optim.Adam,
          optimizer_params={'lr': 2e-5},
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=1000,
          output_path=combine_model_save_path)

# to save time, we use the previously trained model instead of training it everytime

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/359 [00:00<?, ?it/s]

In [61]:
# built-in evaluation

#combine_model_path = '/content/drive/MyDrive/output/training_combined_bert-base-uncased-2021-12-17_15-26-42'
#model = SentenceTransformer(combine_model_path)
model = SentenceTransformer(combine_model_save_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_example, batch_size=16, name='combine-test')
#test_evaluator(model, output_path=combine_model_path)
test_evaluator(model, output_path=combine_model_save_path)

0.7346947862989821

In [62]:
# evaluation using Spearmean Correlation, they coincide with each other
sentencea = model.encode(df_test['sentenceA'])
sentenceb = model.encode(df_test['sentenceB'])
cos_sim = 1 - sklearn.metrics.pairwise.paired_cosine_distances(sentencea, sentenceb)
print(cos_sim)
result = stats.spearmanr(cos_sim, df_test['score'])
print(result)

[0.8419337  0.9905099  0.93246824 ... 0.21353102 0.2472055  0.0803172 ]
SpearmanrResult(correlation=0.7346947862989821, pvalue=8.435239156560313e-234)


# Semantic Search

In [64]:
newspath='/content/drive/MyDrive/dataset/News_Category_Dataset_v2.json'
ss_train = spark.read.json(newspath)
ss_train=ss_train.select('headline')
ss_train.show()
ss_train=ss_train.toPandas()

+--------------------+
|            headline|
+--------------------+
|There Were 2 Mass...|
|Will Smith Joins ...|
|Hugh Grant Marrie...|
|Jim Carrey Blasts...|
|Julianna Margulie...|
|Morgan Freeman 'D...|
|Donald Trump Is L...|
|What To Watch On ...|
|Mike Myers Reveal...|
|What To Watch On ...|
|Justin Timberlake...|
|South Korean Pres...|
|With Its Way Of L...|
|Trump's Crackdown...|
|'Trump's Son Shou...|
|Edward Snowden: T...|
|Booyah: Obama Pho...|
|Ireland Votes To ...|
|Ryan Zinke Looks ...|
|Trump's Scottish ...|
+--------------------+
only showing top 20 rows



In [65]:
newsheadlines = []
for row in ss_train.iloc:
  newsheadlines.append(row['headline'])

In [66]:
#combine_model_path = '/content/drive/MyDrive/output/training_combined_bert-base-uncased-2021-12-17_15-26-42'
combine_model_path = combine_model_save_path
embedder = SentenceTransformer(combine_model_path)
corpus_embeddings = embedder.encode(newsheadlines, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/6277 [00:00<?, ?it/s]

In [68]:
inp_question = input("Please enter a sentence: ")
question_embedding = embedder.encode(inp_question, convert_to_tensor=True)
hits = util.semantic_search(question_embedding, corpus_embeddings)
cos_scores = util.pytorch_cos_sim(question_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=5) # 5 most relative sentences
for score, idx in zip(top_results[0], top_results[1]):
  print(newsheadlines[idx], "(Score: {:.4f})".format(score))

Please enter a sentence: Trump wins the election
Trump Wins The Republican Nomination For President (Score: 0.8536)
Trump Just Proved That Being Famous Is Enough To Win A Primary Election (Score: 0.7686)
Trump Has Won The War Against Sympathy (Score: 0.7479)
Trump's Big Deal (Score: 0.7401)
How Donald Trump Won The Convention Battle (Score: 0.7375)
